In [21]:

import numpy as np
from qulacs import QuantumCircuit
from itertools import combinations
from mcr.equiv_check import (
    equivalence_check_via_mqt_qcec,
    equiv,
    pauli_bit_equivalence_check,
)
from mcr.gate_apply import (
    PauliBit,
    grouping,
    loop_optimization,
    set_clifford_to_qulacs,
    zhang_optimization,
    multiply_all,
)
import pickle

from mcr.mcr_optimize import find_nontrivial_swap, find_mcr
from opt_using_mcr import test_algorithm, mcr_swap, three_layer_nontrivial_swap

In [22]:
nqubits = 2
with open(f"unopt_{nqubits}.pickle", "rb") as f:
    seq = pickle.load(f)
data = []
for elem in seq:
    sgn = str(elem[1])[0]
    pauli_str = str(elem[1])[1:]
    if sgn == "+":
        data.append(PauliBit(pauli_str, np.pi / 4))
    else:
        assert sgn == "-", f"Unexpected sign: {sgn}"
        data.append(PauliBit(pauli_str, -np.pi / 4))
data.append(PauliBit("Z" * nqubits, -np.pi / 4))  # Add identity gate

In [23]:
seq_a = [PauliBit(ele.get_pauli_str(), ele.get_angle()) for ele in data]
clifford_lst, optimized_data = test_algorithm(seq_a, show_opt_log=True)

🎉 Successful optimization using MCR! 48 -> 26
🎉 Successful optimization using MCR! 26 -> 12
🔍 No optimization found in 3th iteration. Try 100 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 99 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 98 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 97 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 96 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 95 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 94 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 93 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 92 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 91 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 90 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 89 times left... 12 -> 12
🔍 No optimization found in 3th iteration. T

何回かループさせないと消せないサンプルが存在する(concatenationと関係？)

In [24]:
tmp = grouping(optimized_data)

In [25]:
tmp

[[PauliBit(-0.7853981633974483*XY)],
 [PauliBit(-0.7853981633974483*IX), PauliBit(-0.7853981633974483*ZI)],
 [PauliBit(0.7853981633974483*XY)],
 [PauliBit(-0.7853981633974483*ZY)],
 [PauliBit(-0.7853981633974483*YY), PauliBit(-0.7853981633974483*ZZ)],
 [PauliBit(0.7853981633974483*IX), PauliBit(0.7853981633974483*XI)],
 [PauliBit(-0.7853981633974483*YZ)],
 [PauliBit(-0.7853981633974483*XZ)],
 [PauliBit(-0.7853981633974483*ZZ)]]

In [26]:
optimized_data

[PauliBit(-0.7853981633974483*XY),
 PauliBit(-0.7853981633974483*IX),
 PauliBit(-0.7853981633974483*ZI),
 PauliBit(0.7853981633974483*XY),
 PauliBit(-0.7853981633974483*ZY),
 PauliBit(-0.7853981633974483*YY),
 PauliBit(-0.7853981633974483*ZZ),
 PauliBit(0.7853981633974483*IX),
 PauliBit(0.7853981633974483*XI),
 PauliBit(-0.7853981633974483*YZ),
 PauliBit(-0.7853981633974483*XZ),
 PauliBit(-0.7853981633974483*ZZ)]

In [7]:
from itertools import permutations
from tqdm import tqdm
i = 0
count = 0
results = []
data_all = tmp[1:]
for i, perm in tqdm(enumerate(permutations(data_all)), desc="Permuting"):
    # if i == 10325:
    target = sum(perm,[])
    cli, tmp2 = test_algorithm(target, show_opt_log=False)
    if equiv([[], sum(data_all, [])], [cli, tmp2]) and len(tmp2) < len(target):
        print("New length:", len(tmp2))
        results.append(perm)
        count += 1
    if count > 10:
        break
        # if len(tmp2) < len(tmp):
        #     tmp = tmp2

Permuting: 312it [00:13, 23.02it/s]


KeyboardInterrupt: 

In [ ]:
tmp[1:]


[[PauliBit(-0.7853981633974483*IX), PauliBit(-0.7853981633974483*ZI)],
 [PauliBit(0.7853981633974483*XY)],
 [PauliBit(-0.7853981633974483*ZY)],
 [PauliBit(-0.7853981633974483*YY), PauliBit(-0.7853981633974483*ZZ)],
 [PauliBit(0.7853981633974483*IX), PauliBit(0.7853981633974483*XI)],
 [PauliBit(-0.7853981633974483*YZ)],
 [PauliBit(-0.7853981633974483*XZ)],
 [PauliBit(-0.7853981633974483*ZZ)]]

In [ ]:
results[0]

([PauliBit(-0.7853981633974483*ZY)],
 [PauliBit(-0.7853981633974483*IX), PauliBit(-0.7853981633974483*ZI)],
 [PauliBit(0.7853981633974483*IX), PauliBit(0.7853981633974483*XI)],
 [PauliBit(0.7853981633974483*XY)],
 [PauliBit(-0.7853981633974483*YY), PauliBit(-0.7853981633974483*ZZ)],
 [PauliBit(-0.7853981633974483*ZZ)],
 [PauliBit(-0.7853981633974483*XZ)],
 [PauliBit(-0.7853981633974483*YZ)])

In [ ]:
tmp[0]

[PauliBit(-0.7853981633974483*XY)]

In [ ]:
grouping(optimized_data)

[[PauliBit(-0.7853981633974483*XY)],
 [PauliBit(-0.7853981633974483*IX), PauliBit(-0.7853981633974483*ZI)],
 [PauliBit(0.7853981633974483*XY)],
 [PauliBit(-0.7853981633974483*ZY)],
 [PauliBit(-0.7853981633974483*YY), PauliBit(-0.7853981633974483*ZZ)],
 [PauliBit(0.7853981633974483*IX), PauliBit(0.7853981633974483*XI)],
 [PauliBit(-0.7853981633974483*YZ)],
 [PauliBit(-0.7853981633974483*XZ)],
 [PauliBit(-0.7853981633974483*ZZ)]]

In [ ]:
equiv([[],seq_a],[clifford_lst, optimized_data])

True

In [ ]:
paulis = ["ix","zi","xy","zy","yy","zz","ix","xi","yz","xz","zz"]
phases = [-1,-1,1,-1,-1,-1,1,1,-1,-1,-1]

paulis_b = ["ix", "zi", "xy", "zy", "ix", "xi", "yy", "zz", "yz", "xz", "zz"]
phases_b = [-1, -1, 1, -1, 1, 1, -1, -1, -1, -1, -1]
# paulis_b = ["zy", "ix", "zi", "ix", "xi", "xy", "yy", "zz", "zz", "xz", "yz"]
# phases_b = [-1, -1, -1, 1, 1, 1, -1, -1, -1, -1, -1]
x = [PauliBit(p, np.pi / 4 * phase) for p, phase in zip(paulis, phases)]

y = [PauliBit(p, np.pi / 4 * phase) for p, phase in zip(paulis_b, phases_b)]
equiv([[], x], [[], y])

True

In [ ]:
cutting_vals = [2,3,4,6,8,9,10]
cands = [
    x[: cutting_vals[0]],
    x[cutting_vals[0] : cutting_vals[1]],
    x[cutting_vals[1] : cutting_vals[2]],
    x[cutting_vals[2] : cutting_vals[3]],
    x[cutting_vals[3] : cutting_vals[4]],
    x[cutting_vals[4] : cutting_vals[5]],
    x[cutting_vals[5] : cutting_vals[6]],
    x[cutting_vals[6] :],
]
cands

[[PauliBit(-0.7853981633974483*IX), PauliBit(-0.7853981633974483*ZI)],
 [PauliBit(0.7853981633974483*XY)],
 [PauliBit(-0.7853981633974483*ZY)],
 [PauliBit(-0.7853981633974483*YY), PauliBit(-0.7853981633974483*ZZ)],
 [PauliBit(0.7853981633974483*IX), PauliBit(0.7853981633974483*XI)],
 [PauliBit(-0.7853981633974483*YZ)],
 [PauliBit(-0.7853981633974483*XZ)],
 [PauliBit(-0.7853981633974483*ZZ)]]

In [ ]:
from itertools import permutations
from tqdm import tqdm

i = 0
count = 0
results = []
parts = cands[5:8]

for i, perm in tqdm(enumerate(permutations(parts)), desc="Permuting"):
    target = sum(perm, [])
    if equiv([[], target], [[], sum(parts, [])]):
        print(f"found!: {i}")
        results.append(perm)
        count += 1
print(f"Total found: {count}")

Permuting: 6it [00:00, 295.26it/s]

found!: 0
Total found: 1


In [ ]:
results

[([PauliBit(-0.7853981633974483*YZ)],
  [PauliBit(-0.7853981633974483*XZ)],
  [PauliBit(-0.7853981633974483*ZZ)])]

In [ ]:
a = [PauliBit("xy", np.pi / 4), PauliBit("zy", -np.pi / 4), PauliBit("yy", -np.pi / 4)]
b = [PauliBit("yy", -np.pi / 4), PauliBit("xy", np.pi / 4), PauliBit("yy", -np.pi / 4)]
equiv([[], a], [[], b])

False

In [8]:
from mcr.gate_apply import set_clifford_to_qulacs
from qulacs.circuit import QuantumCircuitOptimizer as QCO
from qulacs import QuantumCircuit
circ = QuantumCircuit(nqubits)
circ = set_clifford_to_qulacs(circ, clifford_lst)
# for elem in data:
    # circ.merge_circuit(elem.convert_into_qulacs())
QCO().optimize(circ,nqubits)
circ.get_gate(0).get_matrix()

array([[-0.5+0.5j,  0.5-0.5j,  0. +0.j ,  0. +0.j ],
       [-0.5-0.5j, -0.5-0.5j,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j , -0.5-0.5j, -0.5-0.5j],
       [ 0. +0.j ,  0. +0.j ,  0.5-0.5j, -0.5+0.5j]])

In [ ]:
c1 = [("CZ", [0, 1]), ("SWAP", [0, 1]), ("CNOT", [1, 0]), ("H", [1]), ("CNOT", [1, 0])]
c2 = [("CZ", [0, 1]), ("CNOT", [1, 0]), ("H", [1]), ("CNOT", [1, 0])]
c3 = [PauliBit("ZZ", np.pi / 4), PauliBit("ZZ", -np.pi / 4)]
equiv([c1, c3], [c2, c3])  # Should be True

False